In [ ]:
# Run this cell to configure the Google Colab runtime

import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk(".") if "dsait4090_project_location" in files), "."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

Mounted at /content/drive
Google Colab: /content/drive/Othercomputers/macbook_pro/fact-checking


In [ ]:
%load_ext autoreload
%autoreload

from src.early_stopping import EarlyStopping
from src.common import get_device, read_json, DATA_PATH
from src.quantemp_processor import QuantempProcessor, QT_VERACITY_LABELS
from src.models.gpt2 import Gpt2Tokenizer, Gpt2Classifier
from src.classification_training import ClassificationTraining

import time
import torch
import logging
import os
from torch import nn

# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

logging.basicConfig(level=logging.ERROR)

device = get_device()

CUDA: 1, use Tesla T4


In [ ]:
train_data = read_json(f"{DATA_PATH}/raw_data/train_claims_quantemp.json")
val_data = read_json(f"{DATA_PATH}/raw_data/val_claims_quantemp.json")

limit = 20
train_data = train_data[:limit]
val_data = val_data[:limit]

In [ ]:
%autoreload

gpt2_tokenizer = Gpt2Tokenizer()
data_processor = QuantempProcessor(gpt2_tokenizer, claim_decomposition=False)

train_dataset = data_processor.transform(train_data)
val_dataset = data_processor.transform(val_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
%autoreload

model = Gpt2Classifier("gpt2", len(QT_VERACITY_LABELS), mlp_dim=1024, dropout=0.1).to(device)

In [ ]:
%autoreload

training = ClassificationTraining(
    model_name="gpt2/basic",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    model=model,
    optimizer=torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8),
    loss_function=nn.CrossEntropyLoss(),
    batch_size=4,
    early_stopping=EarlyStopping(patience=3),
    device=device,
    random_state=0
)

training.train(epochs=10)

Resuming training from epoch 6

EPOCH 6


train:   0%|          | 0/5 [00:00<?, ?it/s]

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.000
     eval accuracy: 0.800

    avg train loss: 0.937
     avg eval loss: 0.722

Saved checkpoint to ../models/gpt2/basic/checkpoints/06_acc_800

EPOCH 7


train:   0%|          | 0/5 [00:00<?, ?it/s]

...

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.300
     eval accuracy: 0.800

    avg train loss: 0.921
     avg eval loss: 0.727

Saved checkpoint to ../models/gpt2/basic/checkpoints/10_acc_800
Early stopping counter: 2/3

EPOCH 11


train:   0%|          | 0/5 [00:00<?, ?it/s]

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.150
     eval accuracy: 0.800

    avg train loss: 0.868
     avg eval loss: 0.702

Saved checkpoint to ../models/gpt2/basic/checkpoints/11_acc_800
Early stopping counter: 3/3
Saved best model (epoch 2) to: ../models/gpt2/basic/gpt2_basic
